In [19]:
from __future__ import print_function
import cv2
import numpy as np

MAX_MATCHES = 500
GOOD_MATCH_PERCENT = 0.15

In [32]:
def read_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    return img

In [20]:
def convert_to_grayscale(im):
    imGray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    return imGray

In [21]:
def detectORB(MAX_MATCHES):
    orb = cv2.ORB_create(MAX_MATCHES)
    return orb

In [42]:
def compute_descriptors(orb, imGray):
    keypoints, descriptors = orb.detectAndCompute(imGray, None)
    return keypoints, descriptors

In [23]:
def match_features(descriptors1, descriptors2):
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)
    return matches

In [24]:
def sort_matches_by_score(matches):
    matches.sort(key=lambda x: x.distance, reverse=False)
    return matches

In [25]:
def remove_bad_matches(matches,GOOD_MATCH_PERCENT):
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]
    return matches

In [26]:
def draw_top_matches(im1, keypoints1, im2, keypoints2, matches):
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imwrite("matches.jpg", imMatches)
    print("done drawing top matches")

In [71]:
def extract_good_matches_loc(matches, keypoints1, keypoints2):
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
    
    return points1, points2

In [62]:
def find_homography(points1, points2):
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
    return h, mask

In [29]:
def use_homography(im1, im2):
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))
    return im1Reg

In [67]:
def save_aligned_img(img, outFilepath_name):
    print("Saving aligned image : ", outFilepath_name);
    cv2.imwrite(outFilepath_name, img)

In [33]:
 # Read reference image
refFilename = "C:/Users/incha/PycharmProjects/ImageAlignment/form.jpg"
print("Reading reference image : ", refFilename)
imReference = read_image(refFilename)

# Read image to be aligned
imFilename = "C:/Users/incha/PycharmProjects/ImageAlignment/scanned-form.jpg"
print("Reading image to align : ", imFilename);
im = read_image(imFilename)

    

Reading reference image :  C:/Users/incha/PycharmProjects/ImageAlignment/form.jpg
Reading image to align :  C:/Users/incha/PycharmProjects/ImageAlignment/scanned-form.jpg


In [36]:
print(type(im))

<class 'numpy.ndarray'>


In [49]:
im1Gray = convert_to_grayscale(im)
print(type(im1Gray))

<class 'numpy.ndarray'>


In [38]:
im2Gray = convert_to_grayscale(imReference)

In [41]:
orb = detectORB(MAX_MATCHES)
print(type(orb))

<class 'cv2.ORB'>


In [48]:
keypoints1, descriptors1 = compute_descriptors(orb, im1Gray)
print(type(keypoints1))

<class 'list'>


In [50]:
keypoints2, descriptors2 = compute_descriptors(orb, im2Gray)
print(type(keypoints2))
print(type(descriptors2))

<class 'list'>
<class 'numpy.ndarray'>


In [52]:
matches = match_features(descriptors1, descriptors2)
print(type(matches))
print(matches)

<class 'list'>
[<DMatch 00000259801D89B0>, <DMatch 00000259801D8A10>, <DMatch 00000259801D8AB0>, <DMatch 00000259801D86D0>, <DMatch 00000259801D8A50>, <DMatch 00000259801D8AD0>, <DMatch 00000259801D8A70>, <DMatch 00000259801D8A90>, <DMatch 00000259801D8AF0>, <DMatch 00000259801D8B10>, <DMatch 00000259801D8B30>, <DMatch 00000259801D8B50>, <DMatch 00000259801D8B70>, <DMatch 00000259801D8B90>, <DMatch 00000259801D8BB0>, <DMatch 00000259801D8BD0>, <DMatch 00000259801D8BF0>, <DMatch 00000259801D8C10>, <DMatch 00000259801D8C30>, <DMatch 00000259801D8C50>, <DMatch 00000259801D8C70>, <DMatch 00000259801D8C90>, <DMatch 00000259801D8CB0>, <DMatch 00000259801D8CD0>, <DMatch 00000259801D8CF0>, <DMatch 00000259801D8D10>, <DMatch 00000259801D8D30>, <DMatch 00000259801D8D50>, <DMatch 00000259801D8D70>, <DMatch 00000259801D8D90>, <DMatch 00000259801D8DB0>, <DMatch 00000259801D8DD0>, <DMatch 00000259801D8DF0>, <DMatch 00000259801D8E10>, <DMatch 00000259801D8E30>, <DMatch 00000259801D8E50>, <DMatch 0000

In [53]:
matches1 = sort_matches_by_score(matches)
print(type(matches1))
print(matches1)

<class 'list'>
[<DMatch 00000259801D8FD0>, <DMatch 00000259801ED570>, <DMatch 00000259801ECFD0>, <DMatch 00000259801ED610>, <DMatch 00000259801EDC90>, <DMatch 00000259801EDFB0>, <DMatch 00000259801EE850>, <DMatch 00000259801EE930>, <DMatch 00000259801ED7F0>, <DMatch 00000259801ED9F0>, <DMatch 00000259801D8AD0>, <DMatch 00000259801EC2B0>, <DMatch 00000259801EC510>, <DMatch 00000259801EC9D0>, <DMatch 00000259801ED4D0>, <DMatch 00000259801EDCF0>, <DMatch 00000259801EDDF0>, <DMatch 00000259801ED430>, <DMatch 00000259801ED9B0>, <DMatch 00000259801EDA70>, <DMatch 00000259801EC090>, <DMatch 00000259801ED390>, <DMatch 00000259801EDD70>, <DMatch 00000259801EC7B0>, <DMatch 00000259801ED050>, <DMatch 00000259801ED8D0>, <DMatch 00000259801EE4D0>, <DMatch 00000259801EC2F0>, <DMatch 00000259801EC6B0>, <DMatch 00000259801EEC10>, <DMatch 00000259801ED890>, <DMatch 00000259801EE670>, <DMatch 00000259801ED450>, <DMatch 00000259801EEF70>, <DMatch 00000259801EDF30>, <DMatch 00000259801EED50>, <DMatch 0000

In [57]:
matches_good = remove_bad_matches(matches1,GOOD_MATCH_PERCENT)
print(type(matches_good))
print(matches_good)

<class 'list'>
[<DMatch 00000259801D8FD0>, <DMatch 00000259801ED570>, <DMatch 00000259801ECFD0>, <DMatch 00000259801ED610>, <DMatch 00000259801EDC90>, <DMatch 00000259801EDFB0>, <DMatch 00000259801EE850>, <DMatch 00000259801EE930>, <DMatch 00000259801ED7F0>, <DMatch 00000259801ED9F0>, <DMatch 00000259801D8AD0>, <DMatch 00000259801EC2B0>, <DMatch 00000259801EC510>, <DMatch 00000259801EC9D0>, <DMatch 00000259801ED4D0>, <DMatch 00000259801EDCF0>, <DMatch 00000259801EDDF0>, <DMatch 00000259801ED430>, <DMatch 00000259801ED9B0>, <DMatch 00000259801EDA70>, <DMatch 00000259801EC090>, <DMatch 00000259801ED390>, <DMatch 00000259801EDD70>, <DMatch 00000259801EC7B0>, <DMatch 00000259801ED050>, <DMatch 00000259801ED8D0>, <DMatch 00000259801EE4D0>, <DMatch 00000259801EC2F0>, <DMatch 00000259801EC6B0>, <DMatch 00000259801EEC10>, <DMatch 00000259801ED890>, <DMatch 00000259801EE670>, <DMatch 00000259801ED450>, <DMatch 00000259801EEF70>, <DMatch 00000259801EDF30>, <DMatch 00000259801EED50>, <DMatch 0000

In [58]:
draw_top_matches(im, keypoints1, imReference, keypoints2, matches_good)

done drawing top matches


In [72]:
points1, points2 = extract_good_matches_loc(matches_good, keypoints1, keypoints2)

In [61]:
print(type(points2))

<class 'numpy.ndarray'>


In [63]:
h, mask = find_homography(points1, points2)

In [64]:
print(type(h))
print(type(mask))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [65]:
im1Reg = use_homography(im, imReference)

In [66]:
print(type(im1Reg))

<class 'numpy.ndarray'>


In [68]:
outFilepath_name = "C:/Users/incha/PycharmProjects/ImageAlignment/aligned.jpg"
save_aligned_img(im1Reg, outFilepath_name)

Saving aligned image :  C:/Users/incha/PycharmProjects/ImageAlignment/aligned.jpg


In [69]:
print(h)

[[  1.40104964e+00  -2.02959669e-01  -7.41342584e+00]
 [  2.25280216e-01   1.52880342e+00  -4.85162159e+02]
 [ -6.39599712e-05   1.34896590e-04   1.00000000e+00]]
